<a href="https://colab.research.google.com/github/Pavansree14/Python/blob/Pyspark_JSON_metadata_analysis/tes1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3eb8f14ead4d4c03caac3edd43995067212ad1b503e090af5f4b0e0536c2c404
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark import SparkContext, SparkConf

In [6]:
conf = SparkConf().setAppName("testApp1").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [7]:
print(sc)

<SparkContext master=local[*] appName=testApp1>


In [ ]:
import random
randomlist  = random.sample(range(0,40),10)
print(randomlist)

[8, 29, 25, 24, 3, 38, 26, 37, 22, 28]


In [ ]:
rdd1 = sc.parallelize(randomlist, 4)

rdd1.collect()

[8, 29, 25, 24, 3, 38, 26, 37, 22, 28]

In [ ]:
rdd1.getNumPartitions()

4

In [ ]:
print(rdd1.glom().collect())

[[8, 29], [25, 24], [3, 38], [26, 37, 22, 28]]


In [ ]:
rdd_map = rdd1.map(lambda item: (item + 1)*3)
rdd_map.collect()

[27, 90, 78, 75, 12, 117, 81, 114, 69, 87]

In [ ]:
print(rdd_map.glom().collect())

[[27, 90], [78, 75], [12, 117], [81, 114, 69, 87]]


In [ ]:
rdd_filter = rdd1.filter(lambda x: x%3 == 0)
rdd_filter.collect()

[24, 3]

In [ ]:
(rdd_filter.glom().collect())

[[], [24], [3], []]

In [ ]:
#union
print(rdd1.collect())

rdd2 = sc.parallelize([14,20,28,30,24,86,32,22],2)
rdd2.collect()
rdd_union = rdd1.union(rdd2)
print(rdd_union.collect())

print(rdd_union.getNumPartitions())

[8, 29, 25, 24, 3, 38, 26, 37, 22, 28]
[8, 29, 25, 24, 3, 38, 26, 37, 22, 28, 14, 20, 28, 30, 24, 86, 32, 22]
6


In [ ]:
#Intersection
rdd_intersection = rdd1.intersection(rdd2)
print(rdd_intersection.collect())
print(rdd_intersection.getNumPartitions())
rdd_intersection.glom().collect()

[24, 22, 28]
6


[[24], [], [], [], [22, 28], []]

In [ ]:
#find empty partitions
counter = 0
for item in rdd_intersection.glom().collect():
    if len(item) == 0:
      counter = counter + 1

counter


4

In [ ]:
#coalesce(num of partitions)--reduce the size of partitions
rdd_intersection.coalesce(1).glom().collect()

[[24, 22, 28]]

In [ ]:
#takesample(withReplacement, num, [seed])
rdd1.takeSample(False, 5)

[28, 37, 3, 24, 8]

In [ ]:
#takeordered(n, [ordering])
print(rdd1.takeOrdered(5))
print(rdd1.takeOrdered(5, key = lambda x: -x))

[3, 8, 22, 24, 25]
[38, 37, 29, 28, 26]


In [ ]:
rdd2.glom().collect()

[[14, 20, 28, 30], [24, 86, 32, 22]]

In [ ]:
#reduce by key
rdd_rbk = sc.parallelize([(2,6),(7,5),(3,8),(6.8),(2,9),(5,3),(4,6),(6,9)], 2)

print(rdd_rbk.glom().collect())


[[(2, 6), (7, 5), (3, 8), 6.8], [(2, 9), (5, 3), (4, 6), (6, 9)]]


In [ ]:
#reduce
rdd1.reduce(lambda x,y: x-y)

74

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Read and Load Data to Spark
import json

rdd_json = sc.textFile("/content/drive/MyDrive/Colab Notebooks/arxiv-metadata-oai-snapshot.json",100)
rdd = rdd_json.map(lambda x: json.loads(x))
rdd.persist()

PythonRDD[9] at RDD at PythonRDD.scala:53

Query optimization
catalyst query optimization
Project tungsten.

In [14]:
#Count of elements
rdd.count()

2011231

In [15]:
#First two records
rdd.take(2)

[{'id': '0704.0001',
  'submitter': 'Pavel Nadolsky',
  'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
  'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
  'comments': '37 pages, 15 figures; published version',
  'journal-ref': 'Phys.Rev.D76:013009,2007',
  'doi': '10.1103/PhysRevD.76.013009',
  'report-no': 'ANL-HEP-PR-07-12',
  'categories': 'hep-ph',
  'license': None,
  'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with d

In [18]:
# How to get all the distinct attributes
rdd.flatMap(lambda x: x.keys()).distinct().collect()

['authors',
 'comments',
 'title',
 'id',
 'journal-ref',
 'versions',
 'submitter',
 'categories',
 'update_date',
 'authors_parsed',
 'report-no',
 'license',
 'abstract',
 'doi']

In [22]:
#Get the name of the license
rdd.map(lambda x: x["license"]).distinct().collect()

[None,
 'http://creativecommons.org/licenses/publicdomain/',
 'http://creativecommons.org/licenses/by-nc-nd/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/4.0/',
 'http://creativecommons.org/licenses/by-nc-sa/3.0/',
 'http://creativecommons.org/licenses/by/3.0/',
 'http://creativecommons.org/licenses/by/4.0/',
 'http://creativecommons.org/publicdomain/zero/1.0/',
 'http://arxiv.org/licenses/nonexclusive-distrib/1.0/',
 'http://creativecommons.org/licenses/by-sa/4.0/']